# Finding the Non-personalized Recomendation:
Finding all pairs of movies
In this exercise, you will work through how to find all pairs of movies or all permutations of pairs of movies that have been watched by the same person.

The user_ratings_df has been loaded once again containing users, and the movies they have seen.

You will need to first create a function that finds all possible pairs of items in a list it is applied to. For ease of use, you will output the values of this as a DataFrame. Since you only want to find movies that have been seen by the same person and not all possible permutations, you will group by user_id when applying the function.

In [ ]:
from itertools import permutations

# Create the function to find all permutations
def find_movie_pairs(x):
  pairs = pd.DataFrame(list(permutations(x.values, 2)),
                       columns=['movie_a', 'movie_b'])
  return pairs

# Apply the function to the title column and reset the index
movie_combinations = user_ratings_df.groupby('userId')['title'].apply(find_movie_pairs).reset_index(drop=True)

print(movie_combinations)

##  Counting up the pairs
You can now create DataFrame of all the permutations of movies that have been watched by the same user. This is of limited use unless you can find which movies are most commonly paired.

In this exercise, you will work with the movie_combinations DataFrame that you created in the last exercise (that has been loaded for you), and generate a new DataFrame containing the counts of occurrences of each of the pairs within.

In [ ]:
# Calculate how often each item in movie_a occurs with the items in movie_b
combination_counts = movie_combinations.groupby(['movie_a', 'movie_b']).size()

# Convert the results to a DataFrame and reset the index
combination_counts_df = combination_counts.to_frame(name='size').reset_index()
print(combination_counts_df.head())

In [ ]:
          movie_a |                         movie_b | size
0  21 Jump Street |          Atlas Shrugged: Part 1 |    1
1  21 Jump Street | Avengers: Infinity War - Part I |    3
2  21 Jump Street |                        Bad Boys |    7
3  21 Jump Street |                    Bad Teacher  |    8
4  21 Jump Street |             Battle: Los Angeles |    2

## Making your first movie recommendations
Now that you have found the most commonly paired movies, you can make your first recommendations!

While you are not taking in any information about the person watching, and do not even know any details about the movie, valuable recommendations can still be made by examining what groups of movies are watched by the same people. In this exercise, you will examine the movies often watched by the same people that watched Thor, and then use this data to give a recommendation to someone who just watched the movie. The DataFrame you generated in the last lesson, combination_counts_df, that contains counts of how often movies are watched together has been loaded for you.

In [ ]:
import matplotlib.pyplot as plt

# Sort the counts from highest to lowest
combination_counts_df.sort_values('size', ascending=False, inplace=True)

# Find the movies most frequently watched by people who watched Thor
thor_df = combination_counts_df[combination_counts_df['movie_a'] == 'Thor']

# Plot the results
thor_df.plot.bar(x="movie_b")
plt.show()

In [ ]:
In [2]:
thor_df
Out[2]:

    movie_a                          movie_b  size
137    Thor                   21 Jump Street    12
147    Thor                    Green Lantern    10
143    Thor                      Bridesmaids     9
140    Thor                         Bad Boys     8
141    Thor                      Bad Teacher     6
139    Thor  Avengers: Infinity War - Part I     6
142    Thor              Battle: Los Angeles     4
146    Thor                           Cars 2     2
145    Thor                          Carnage     2
144    Thor                  Captain America     2
148    Thor                              Rio     2
138    Thor           Atlas Shrugged: Part 1     1

## Inference - 
Good work! You can see that 21 Jump Street was the most commonly watched movie by those who watched Thor. This means that it would be a good movie to recommend Thor watchers as it shows they have similar fans.

# Content Based Recomendation 

In [ ]:
# Select only the rows with values in the name column equal to Toy Story
toy_story_genres = movie_genre_df[movie_genre_df['name'] == 'Toy Story']


In [ ]:
"""
movie_genre_df.head(2)


        name genre_list
0  Toy Story  Adventure
1  Toy Story  Animation
"""

In [ ]:
# Create cross-tabulated DataFrame from name and genre_list columns
movie_cross_table = pd.crosstab(movie_genre_df['name'], movie_genre_df['genre_list'])

In [ ]:
"""
movie_cross_table


genre_list                      Action  Adventure  Animation  Children  Comedy  ...  Drama  Fantasy  Horror  Romance  Thriller
name                                                                            ...                                           
Ace Ventura: When Nature Calls       0          0          0         0       1  ...      0        0       0        0         0
American President, The              0          0          0         0       1  ...      1        0       0        1         0
Balto                                0          1          1         1       0  ...      0        0       0        0         0
"""

In [ ]:
# Select only the rows with Toy Story as the index
toy_story_genres_ct = movie_cross_table[movie_cross_table.index == 'Toy Story']
print(toy_story_genres_ct)

In [ ]:
"""
In [7]:
toy_story_genres_ct
Out[7]:

genre_list  Action  Adventure  Animation  Children  Comedy  ...  Drama  Fantasy  Horror  Romance  Thriller
name                                                        ...                                           
Toy Story        0          1          1         1       1  ...      0        1       0        0         0

"""

## Comparing individual movies with Jaccard similarity
In the last lesson, you built a DataFrame of movies, where each column represents a different genre. You can now use this DataFrame to compare movies by measuring the Jaccard similarity between rows. The higher the Jaccard similarity score, the more similar the two items are.

In this exercise, you will compare the movie GoldenEye with the movie Toy Story, and GoldenEye with SkyFall and compare the results.

The DataFrame movie_cross_table containing all the movies as rows and the genres as Boolean columns that you created in the last lesson has been loaded.

In [ ]:
# Import numpy and the distance metric
import numpy as np
from sklearn.metrics import jaccard_score

# Extract just the rows containing GoldenEye and Toy Story
goldeneye_values = movie_cross_table.loc['GoldenEye'].values
toy_story_values = movie_cross_table.loc['Toy Story'].values

# Find the similarity between GoldenEye and Toy Story
print(jaccard_score(goldeneye_values, toy_story_values))

# Repeat for GoldenEye and Skyfall
skyfall_values = movie_cross_table.loc['Skyfall'].values
print(jaccard_score(goldeneye_values, skyfall_values))

## Comparing all your movies at once
While finding the Jaccard similarity between any two individual movies in your dataset is great for small-scale analyses, it can prove slow on larger datasets to make recommendations.

In this exercise, you will find the similarities between all movies and store them in a DataFrame for quick and easy lookup.

When finding the similarities between the rows in a DataFrame, you could run through all pairs and calculate them individually, but it's more efficient to use the pdist() (pairwise distance) function from scipy.

This can be reshaped into the desired rectangular shape using squareform() from the same library. Since you want similarity values as opposed to distances, you should subtract the values from 1.

movie_cross_table has once again been loaded for you.

In [ ]:
# Import functions from scipy
from scipy.spatial.distance import pdist, squareform

# Calculate all pairwise distances
jaccard_distances = pdist(movie_cross_table.values, metric='jaccard')

# Convert the distances to a square matrix
jaccard_similarity_array = 1 -  squareform(jaccard_distances)

# Wrap the array in a pandas DataFrame
jaccard_similarity_df = pd.DataFrame(jaccard_similarity_array, index=movie_cross_table.index, columns=movie_cross_table.index)

# Print the top 5 rows of the DataFrame
print(jaccard_similarity_df.head())

In [ ]:
'''

____ Output :: 

name                                  21 Jump Street  Alvin and the Chipmunks: Chipwrecked  Another Earth  Beastly  Bridesmaids  ...    Cars 2  Green Lantern  Oldboy       Rio      Thor
name                                                                                                                             ...                                                     
21 Jump Street                              1.000000                                  0.25            0.0      0.0     0.333333  ...  0.142857            0.2     0.2  0.166667  0.142857
Alvin and the Chipmunks: Chipwrecked        0.250000                                  1.00            0.0      0.0     0.500000  ...  0.400000            0.0     0.0  0.500000  0.000000
Another Earth                               0.000000                                  0.00            1.0      0.5     0.000000  ...  0.000000            0.2     0.2  0.000000  0.142857
Beastly                                     0.000000                                  0.00            0.5      1.0     0.000000  ...  0.000000            0.0     0.2  0.000000  0.333333
Bridesmaids                                 0.333333                                  0.50            0.0      0.0     1.000000  ...  0.200000            0.0     0.0  0.250000  0.000000

'''

In [ ]:
## Recommend a Movie similar to Thor 
# Wrap the preloaded array in a DataFrame
jaccard_similarity_df = pd.DataFrame(jaccard_similarity_array, index=movie_cross_table.index, columns=movie_cross_table.index)

# Find the values for the movie Thor
jaccard_similarity_series = jaccard_similarity_df.loc['Thor']

# Sort these values from highest to lowest
ordered_similarities = jaccard_similarity_series.sort_values(ascending=False)

# Print the results
print(ordered_similarities)

In [ ]:
'''
name
Thor                                    1.000000
Green Lantern                           0.333333
Cars 2                                  0.250000
Captain America: The First Avenger      0.250000
Carnage                                 0.166667
Another Earth                           0.142857
21 Jump Street                          0.142857
Rio                                     0.125000
Bridesmaids                             0.000000
Alvin and the Chipmunks: Chipwrecked    0.000000

'''

## Content-Based using Text Features 

In [ ]:
'''
df_plots


                             Title                                               Plot
0   Ace Ventura: When Nature Calls  In the Himalayas, after a failed rescue missio...
1      Dracula: Dead and Loving It  Solicitor Thomas Renfield travels all the way ...
2      Father of the Bride Part II  The film begins five years after the events of...
3                       Four Rooms  The film is set on New Year's Eve, and starts ...
4                 Grumpier Old Men  The feud between Max (Walter Matthau) and John...
5                          Jumanji  In 1869, near Brantford, New Hampshire, two br...
6                     Sudden Death  Darren McCord (Jean-Claude Van Damme) is a Fre...
7                     Tom and Huck  The movie opens with Injun Joe (Eric Schweig) ...
8                        Toy Story  In a world where toys are living things who pr...
9                Waiting to Exhale  "Friends are the People who let you be yoursel...
10                       GoldenEye  In 1986, at Arkhangelsk, MI6 agents James Bond...
11                         Skyfall  MI6 agents James Bond and Eve Moneypenny pursu...
'''

In [ ]:
rom sklearn.feature_extraction.text import TfidfVectorizer

# Instantiate the vectorizer object and transform the plot column
vectorizer = TfidfVectorizer(max_df=0.7, min_df=2)
vectorized_data = vectorizer.fit_transform(df_plots['Plot']) 

# Create Dataframe from TF-IDFarray
tfidf_df = pd.DataFrame(vectorized_data.toarray(), columns=vectorizer.get_feature_names())

# Assign the movie titles to the index and inspect
tfidf_df.index = df_plots['Title']
print(tfidf_df.head())

In [ ]:
'''
                               000       100  abandoned     above  accidentally  ...     wrong      year     years       you     young
Title                                                                                  ...                                                  
Ace Ventura: When Nature Calls  0.000000  0.000000        0.0  0.000000      0.000000  ...  0.000000  0.000000  0.044595  0.000000  0.053863
Dracula: Dead and Loving It     0.000000  0.000000        0.0  0.000000      0.000000  ...  0.000000  0.000000  0.000000  0.055645  0.000000
Father of the Bride Part II     0.045850  0.045850        0.0  0.000000      0.000000  ...  0.045850  0.000000  0.030099  0.000000  0.072708
Four Rooms                      0.039916  0.039916        0.0  0.079831      0.039916  ...  0.039916  0.079831  0.026203  0.000000  0.000000
Grumpier Old Men                0.000000  0.000000        0.0  0.000000      0.000000  ...  0.000000  0.000000  0.000000  0.000000  0.000000


'''

In [ ]:
# Import cosine_similarity measure
from sklearn.metrics.pairwise import cosine_similarity

# Create the array of cosine similarity values
cosine_similarity_array = cosine_similarity(tfidf_summary_df)

# Wrap the array in a pandas DataFrame
cosine_similarity_df = pd.DataFrame(cosine_similarity_array, columns=tfidf_summary_df.index, index=tfidf_summary_df.index)

# Print the top 5 rows of the DataFrame
print(cosine_similarity_df.head()) ## 18 * 18 matrix


In [ ]:
'''
                                                       Thor  21 Jump Street  The Avengers    Oldboy  
The Adventures of Tintin: The Secret of the Uni...  0.312927        0.282663      0.374425  0.248183  
Alvin and the Chipmunks: Chipwrecked                0.323938        0.311788      0.400024  0.267687  
Another Earth                                       0.304739        0.236896      0.229218  0.249804  
Beastly                                             0.229194        0.187408      0.186539  0.207715  
The Beaver                                          0.300383        0.238325      0.266592  0.253751  


'''

In [ ]:
## How to make Recommendation 
# Wrap the preloaded array in a DataFrame
cosine_similarity_df = pd.DataFrame(cosine_similarity_array, index=tfidf_summary_df.index, columns=tfidf_summary_df.index)

# Find the values for the movie Thor
cosine_similarity_series = cosine_similarity_df.loc['Rio']

# Sort these values highest to lowest
ordered_similarities = cosine_similarity_series.sort_values(ascending=False)

# Print the results
print(ordered_similarities)

In [ ]:
'''
Rio                                                    1.000000
Alvin and the Chipmunks: Chipwrecked                   0.361180
The Avengers                                           0.344869
The Hangover: Part II                                  0.344407
The Adventures of Tintin: The Secret of the Unicorn    0.327422
Thor                                                   0.318216
Green Lantern                                          0.314570
Carnage                                                0.312067
Cars 2                                                 0.306499
21 Jump Street                                         0.290252
Another Earth                                          0.281780
Captain America: The First Avenger                     0.266358
The Twilight Saga: Breaking Dawn - Part 1              0.262755
Oldboy                                                 0.252323
Bridesmaids                                            0.234514
Beastly                                                0.213502

'''